In [18]:
name = "박성남"

f'my name is {name}'

SyntaxError: invalid syntax (<ipython-input-18-5f37f460b52b>, line 3)

In [27]:
class foo:
    def __init__(self, var1: int):
        self.var = var1
    

In [30]:
f = foo("s")

In [31]:
f.var

's'